In [61]:
import pandas as pd

### A partir d'un dataFrame cette methode va récuperer toute les cartes joués et en faire des phrases qui seront en suite passé à l'algorithme word2vec

### A games 
### On prend la colonne games

### A partir de card_history
### on recupere l'historique des cartes sorties pour chaque partie On filtre si le champs card_history existe ou si il est vide

### A partir de sentances
### Pour chaque partie on rassemble toutes les cartes sorties par "me" et "opponent", puis on recupère le nom des cartes pour crée des phrases.


In [ ]:
def createsentances(data):
    games = data["games"]
    cards = [e["card_history"] for e in games if e.has_key("card_history") and e["card_history"]!=[]]
    sentances = []
    for c in cards:
        sentances.append([e["card"]["name"]for e in c if e["player"] == "me"])
        sentances.append([e["card"]["name"]for e in c if e["player"] == "opponent"])
    return sentances

creation de phrase a partir de plusieurs fichier json
listfile est une liste de fichier avec leur chemin

In [ ]:
def createsentancesfiles(listfile):
    sentances = []
    for l in listfile:
        data=pd.read_json(l)
        sentances += createsentances(data)
        del data
    return sentances

In [ ]:
directoryHearthstoneGames = "HeathstoneGames/"
from os import listdir
from os.path import isfile, join
filesHearthstoneGames = [directoryHearthstoneGames+f for f in listdir(directoryHearthstoneGames) if isfile(join(directoryHearthstoneGames, f))]


In [ ]:
sentances = createsentancesfiles(filesHearthstoneGames)

In [62]:
data = pd.read_json("2018-01.zip");

In [ ]:
sentances = createsentances(data)


# Je prend l'url avec collectible car je ne veux que les cartes disponibles pour faire un deck

In [39]:
import requests

urlApi = "https://api.hearthstonejson.com/v1/latest/enUS/cards.collectible.json"

ApiHearthStone = requests.get(urlApi).json()

In [6]:
listNameCard = [e["name"] for e in ApiHearthStone]

on filtre les cartes des parties à seulement celle disponible pour la construction de deck

In [ ]:
sentances = [filter(lambda x: x in listNameCard, e) for e in sentances]

# On va utilisier gensim et plus particulièrement l'algo Word2vec

In [5]:
from gensim.models import Word2Vec

max_count exclue les mots dont leur nombre d'apparition est inférieure à celui indiqué
size nombre de noeuds
workers paralléliser sur un nombres de coeurs
ier nombre d'epochs

In [ ]:
model = Word2Vec(sentances,min_count=10,size=200,workers=4,iter=100)

sauvegarde du model

In [ ]:
model.save("word2vecModel")

charger le modele entrainer et de continuer l'entrainement

In [6]:
model = Word2Vec.load("word2vecModel")

In [53]:
s =[]
for e,_ in t: 
  s +=[i["cardClass"] for i in ApiHearthStone if i["name"]==e]
s

[u'PALADIN', u'DRUID', u'DRUID', u'DRUID', u'PALADIN']

In [ ]:
model.wv.vocab.keys()

proposition de cartes si deck 30 ou plus aucune proposition

negative si cardClass différent 
meme classes de carte que le heros sauf neutral
regler probleme cartes cardClass diférente

In [7]:
def proposeCards(uncompletedDeck,model,nbcartes=2):
    cardClass = ""
    uncompletedDeckInside =  list(uncompletedDeck)
    numberMaxCards = 30
    if nbcartes>10:
        nbcartes = 10
    if len(uncompletedDeckInside) >= numberMaxCards:
        return
      
    while len(uncompletedDeckInside)<numberMaxCards:
        proposes = model.wv.most_similar(positive=uncompletedDeckInside,topn=5)
        diff = numberMaxCards-len(uncompletedDeckInside)
        if diff < nbcartes :
            p = proposes[:diff]
        else: 
            p = proposes[:nbcartes]
        
        p = [x[0] for x in p]
        uncompletedDeckInside += p  
    return uncompletedDeckInside

In [26]:
myuncompletedDeck = ["Swipe","Equality"]

#decks = proposeCards(myuncompletedDeck,model,nbcartes=4)
decks=[]
for i in range(1,6):
    decks.append(proposeCards(myuncompletedDeck,model,nbcartes=i))

In [42]:
[i["cardClass"] for i in ApiHearthStone if i["name"] in decks[1]]

[u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'DRUID',
 u'DRUID',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'DRUID',
 u'DRUID',
 u'DRUID',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'DRUID',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN',
 u'PALADIN']

In [34]:
decks[1]

['Swipe',
 'Equality',
 u'Aldor Peacekeeper',
 u'Innervate',
 u'Consecration',
 u'Wild Growth',
 u'Wrath',
 u'Truesilver Champion',
 u'Tirion Fordring',
 u'Feral Rage',
 u'Ragnaros, Lightlord',
 u'Nourish',
 u'Forbidden Healing',
 u'Ivory Knight',
 u'Wickerflame Burnbristle',
 u'Solemn Vigil',
 u'Lay on Hands',
 u'Humility',
 u'Spikeridged Steed',
 u'Uther of the Ebon Blade',
 u'Grimestreet Protector',
 u'Hammer of Wrath',
 u'Keeper of Uldaman',
 u'Rallying Blade',
 u'Sunkeeper Tarim',
 u'Repentance',
 u'Redemption',
 u'Blessing of Kings',
 u'Noble Sacrifice',
 u'Getaway Kodo']

cardClassDecide = []
    for e in uncompletedDeck:
       cardClassDecide += [i["cardClass"] for i in apiHS if i["name"]==e]